## Load dataset

In [1]:
from huggingface_hub import login
import sys
from datasets import load_dataset
from transformers import AutoTokenizer

# Load your CSV dataset
# The dataset should have two columns: "vulnerable_code" and "fixed_code"
dataset = load_dataset("json", data_files="MSR_data_cleaned.json")
# Split the dataset into training and validation sets
#dataset = dataset["train"].train_test_split(test_size=0.2)
print('test')
# Initialize the tokenizer for Llama
tokenizer = AutoTokenizer.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded")  # Replace with your desired Llama model
print('test2')
# Tokenize the data

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


test
test2


In [9]:
print(len(dataset["train"][0]))

<class 'dict'>


In [5]:
def tokenize_function(example):
    print(len(example))
    # Prepare input as "Fix the vulnerability: [vulnerable_code]" and target as "fixed_code"
    example["input_text"] = f"Fix the vulnerability: {example['func_before']}"
    example["target_text"] = example["vul_func_with_fix"]
    
    # Tokenize input and target
    input_encodings = tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=512)
    target_encodings = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=512)

    # Return the encodings as input_ids and labels
    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }


In [6]:
tokenized_dataset = dataset['train'].map(tokenize_function, batched=True)
print('test3')
# Set the dataset format for PyTorch
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

188636


KeyError: 'func_before'